In [21]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [22]:
model_name = 'faceDetectModelAugmentation'
savedModelPath = '../log/saved_models/{model_name}'
tb_log_dir = "../log/tensorboard/{model_name}"
cp_filepath = '../log/cps/{model_name}/latest_weights.h5'

batch_size = 32
epochs = 5

doDataAugmentation = True

## Model callbacks

In [23]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
                    # Checkpoint callback                    
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

                    # Tensorboard callback
                    tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)
            ]

## Data parameters

In [24]:
image_height = 224
image_width = 224

## Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)

## Creating the training dataset

In [25]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 0 files belonging to 0 classes.


ValueError: When passing `label_mode="binary"`, there must be exactly 2 class_names. Received: class_names=[]

## Creating the validation dataset

In [ ]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

Found 28312 files belonging to 2 classes.
Using 5662 files for validation.


## Creating the test dataset

In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    "../data/test/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

## Load the ResNet model

In [ ]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
resnet = ResNet50(
                include_top=False,
                weights=None,  # No pretrained weights
                input_shape=(image_height, image_width, 3)
                )

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

## Adding Pooling and Fully Connected Layer for binary classification

In [ ]:
inputs = keras.Input(shape=(image_height, image_width, 3))

if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
x = resnet(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.Flatten()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

model.summary()

# Loading latest saved weights into model
# if(os.path.isfile(cp_filepath)):
#     print('Loading weights...')
#     model.load_weights(filepath=cp_filepath)

## Training the model with the dataset

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=epochs, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
708/708 [==============================] - ETA: 0s - loss: 6.5250 - accuracy: 0.7446
Epoch 1: saving model to ../log/cps/faceDetectModel\latest_weights.h5
708/708 [==============================] - 2766s 4s/step - loss: 6.5250 - accuracy: 0.7446 - val_loss: 0.3156 - val_accuracy: 0.8864
Epoch 2/10
708/708 [==============================] - ETA: 0s - loss: 0.2908 - accuracy: 0.8826
Epoch 2: saving model to ../log/cps/faceDetectModel\latest_weights.h5
708/708 [==============================] - 2752s 4s/step - loss: 0.2908 - accuracy: 0.8826 - val_loss: 0.2334 - val_accuracy: 0.9218
Epoch 3/10
708/708 [==============================] - ETA: 0s - loss: 0.2371 - accuracy: 0.9057
Epoch 3: saving model to ../log/cps/faceDetectModel\latest_weights.h5
708/708 [==============================] - 2746s 4s/step - loss: 0.2371 - accuracy: 0.9057 - val_loss: 0.2174 - val_accuracy: 0.9082
Epoch 4/10
708/708 [==============================] - ETA: 0s - loss: 0.2047 - accuracy: 0.9216
Epoch 4

## Save the model

In [ ]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../log/saved_models/faceDetectModel\assets


INFO:tensorflow:Assets written to: ../log/saved_models/faceDetectModel\assets


## Test model with test dataset

In [ ]:
results = model.evaluate(test_ds)

print(f'Loss: {results[0]}; Accuracy: {results[1]}')

## Predict new images

In [34]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/NoFace/antelope_10248.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))

1/1 [==============================] - 0s 99ms/step
This image contains a face. (99.00 percent confidence)
